In [4]:
%pip install -U scikit-learn
%pip install -U python-dotenv



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:


import json
import os
from typing import List, Dict, Tuple
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
from dotenv import load_dotenv
import re

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
class DataLoader:
    """Load và quản lý dữ liệu từ 6 file JSON"""
    
    def __init__(self):
        self.disease_names = {}
        self.docs = []
        self.metadata = {}
        self.symptom_dict = {}
        self.qa_bank_flat = []
        self.qa_bank_full = []
    
    def load_from_strings(self, disease_json: str, docs_json: str, 
                         meta_json: str, symptom_json: str,
                         qa_flat_json: str, qa_full_json: str):
        self.disease_names = json.loads(disease_json)
        self.docs = json.loads(docs_json) if isinstance(json.loads(docs_json), list) else [json.loads(docs_json)]
        self.metadata = json.loads(meta_json)
        self.symptom_dict = json.loads(symptom_json)
        self.qa_bank_flat = json.loads(qa_flat_json) if isinstance(json.loads(qa_flat_json), list) else [json.loads(qa_flat_json)]
        self.qa_bank_full = json.loads(qa_full_json) if isinstance(json.loads(qa_full_json), list) else [json.loads(qa_full_json)]
        
        print(f"✓ Loaded {len(self.disease_names)} diseases")
        print(f"✓ Loaded {len(self.docs)} documents")
        print(f"✓ Loaded {len(self.qa_bank_flat)} QA pairs")
        print(f"✓ Loaded {len(self.qa_bank_full)} disease entries")

data_loader = DataLoader()

In [3]:
class EmbeddingGenerator:
    """Tạo embeddings sử dụng OpenAI"""
    
    def __init__(self, client):
        self.client = client
        self.embedding_cache = {}
    
    def get_embedding(self, text: str, model="text-embedding-3-small") -> List[float]:
        """Lấy embedding cho text"""
        if text in self.embedding_cache:
            return self.embedding_cache[text]
        
        text = text.replace("\n", " ").strip()
        response = self.client.embeddings.create(input=[text], model=model)
        embedding = response.data[0].embedding
        self.embedding_cache[text] = embedding
        return embedding
    
    def compute_similarity(self, emb1: List[float], emb2: List[float]) -> float:
        """Tính cosine similarity"""
        return cosine_similarity([emb1], [emb2])[0][0]

embedding_gen = EmbeddingGenerator(client)
print("✓ Embedding generator initialized")

✓ Embedding generator initialized


In [4]:
class IntentClassifier:
    """Phân loại intent của user query"""
    
    INTENTS = {
        "symptoms": ["triệu chứng", "dấu hiệu", "biểu hiện", "symptoms", "signs"],
        "herb": ["thảo dược", "nghệ", "đông y", "yhct", "herb", "herbal"],
        "drug": ["thuốc tây", "thuốc", "drug", "medicine", "medication"],
        "interaction": ["tương tác", "dùng chung", "kết hợp", "interaction", "combine"],
        "symptom_to_disease": ["tôi bị", "tôi có", "bệnh gì", "what disease", "I have"],
        "general": ["là gì", "what is", "thông tin", "information"]
    }
    
    def __init__(self, client):
        self.client = client
    
    def classify(self, query: str) -> str:
        """Phân loại intent dựa trên keywords và LLM"""
        query_lower = query.lower()
        
        for intent, keywords in self.INTENTS.items():
            for keyword in keywords:
                if keyword in query_lower:
                    return intent
        
        return self._classify_with_llm(query)
    
    def _classify_with_llm(self, query: str) -> str:
        """Sử dụng LLM để phân loại intent phức tạp"""
        prompt = f"""Phân loại câu hỏi sau vào một trong các intent:
- symptoms: Hỏi về triệu chứng của bệnh
- herb: Hỏi về thảo dược, đông y
- drug: Hỏi về thuốc tây y
- interaction: Hỏi về tương tác thuốc-thảo dược
- symptom_to_disease: Mô tả triệu chứng và hỏi bệnh gì
- general: Hỏi chung về bệnh

Câu hỏi: "{query}"

Trả lời ĐÚNG MỘT TỪ trong các intent trên:"""
        
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=10
        )
        
        intent = response.choices[0].message.content.strip().lower()
        return intent if intent in self.INTENTS.keys() else "general"

intent_classifier = IntentClassifier(client)
print("✓ Intent classifier initialized")

✓ Intent classifier initialized


In [5]:
class RAGRetriever:
    
    def __init__(self, data_loader, embedding_gen):
        self.data_loader = data_loader
        self.embedding_gen = embedding_gen
        self.qa_embeddings = []
        self._build_index()
    
    def _build_index(self):
        """Build embedding index cho QA bank"""
        print("Building QA embeddings index...")
        for qa in self.data_loader.qa_bank_flat:
            questions_text = " ".join(qa.get("questions", []))
            embedding = self.embedding_gen.get_embedding(questions_text)
            self.qa_embeddings.append({
                "qa": qa,
                "embedding": embedding
            })
        print(f"✓ Built index with {len(self.qa_embeddings)} QA pairs")
    
    def retrieve(self, query: str, intent: str, top_k: int = 3) -> List[Dict]:
        """Retrieve top-k relevant QA pairs"""
        query_embedding = self.embedding_gen.get_embedding(query)
        
        scores = []
        for item in self.qa_embeddings:
            if intent != "general" and item["qa"].get("intent") != intent:
                continue
            
            similarity = self.embedding_gen.compute_similarity(
                query_embedding, 
                item["embedding"]
            )
            scores.append((similarity, item["qa"]))
        
        scores.sort(reverse=True, key=lambda x: x[0])
        return [{"score": score, "qa": qa} for score, qa in scores[:top_k]]
    
    def get_disease_info(self, disease_name: str) -> Dict:
        """Lấy thông tin đầy đủ về bệnh"""
        for disease_entry in self.data_loader.qa_bank_full:
            disease = disease_entry.get("disease", {})
            if disease.get("label") == disease_name or \
               disease.get("vi") == disease_name or \
               disease.get("en") == disease_name:
                return disease_entry
        return None

rag_retriever = None  #
print("✓ RAG retriever class defined")

✓ RAG retriever class defined


In [7]:
class ResponseGenerator:
    """Generate câu trả lời sử dụng LLM"""
    
    def __init__(self, client, data_loader):
        self.client = client
        self.data_loader = data_loader
    
    def generate(self, query: str, intent: str, retrieved_docs: List[Dict]) -> str:
        """Generate response từ retrieved documents"""
        
        if not retrieved_docs:
            return self._generate_no_info_response(query)
        
        context = self._prepare_context(retrieved_docs)
        
        system_prompt = """Bạn là trợ lý y tế AI chuyên nghiệp, trả lời HOÀN TOÀN dựa trên dữ liệu có sẵn trong cơ sở dữ liệu (CSDL).

QUY TẮC BẮT BUỘC:
1. CHỈ sử dụng thông tin từ CSDL được cung cấp
2. KHÔNG bổ sung kiến thức y khoa bên ngoài
3. KHÔNG đưa ra khuyến nghị điều trị cụ thể
4. Luôn nhắc nhở: "⚠️ Thông tin mang tính tham khảo, không thay thế chẩn đoán và tư vấn y khoa trực tiếp"
5. Nếu CSDL không có thông tin, nói rõ "CSDL chưa có thông tin về..."
6. Trích dẫn nguồn từ CSDL khi có
7. Trả lời bằng tiếng Việt, rõ ràng, có cấu trúc

ĐỊNH DẠNG TRẢ LỜI:
- Sử dụng bullet points khi liệt kê
- Highlight từ khóa quan trọng bằng **bold**
- Thêm emoji phù hợp để dễ đọc
"""
        
        user_prompt = f"""Câu hỏi: {query}

Intent: {intent}

Thông tin từ CSDL:
{context}

Hãy trả lời câu hỏi dựa HOÀN TOÀN trên thông tin từ CSDL trên."""
        
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
            max_tokens=1500
        )
        
        return response.choices[0].message.content
    
    def _prepare_context(self, retrieved_docs: List[Dict]) -> str:
        """Chuẩn bị context từ retrieved documents"""
        context_parts = []
        
        for i, doc in enumerate(retrieved_docs, 1):
            qa = doc["qa"]
            score = doc["score"]
            
            context_parts.append(f"""
[Tài liệu {i}] (Độ liên quan: {score:.2f})
Bệnh: {qa.get('disease', 'N/A')}
Intent: {qa.get('intent', 'N/A')}
Câu trả lời từ CSDL:
{qa.get('answer', 'N/A')}
---
""")
        
        return "\n".join(context_parts)
    
    def _generate_no_info_response(self, query: str) -> str:
        """Generate response khi không tìm thấy thông tin"""
        return f"""Xin lỗi, CSDL hiện tại chưa có đủ thông tin để trả lời câu hỏi: "{query}"

 Gợi ý:
- Thử diễn đạt câu hỏi khác đi
- Cung cấp thêm triệu chứng cụ thể
- Nêu rõ tên bệnh hoặc thuốc bạn muốn hỏi

 Nếu bạn có triệu chứng nghiêm trọng, hãy đến cơ sở y tế để được thăm khám trực tiếp.
"""

response_generator = None  
print("✓ Response generator class defined")

✓ Response generator class defined


In [8]:
class MedicalChatbot:
    """Main chatbot class kết hợp tất cả components"""
    
    def __init__(self, client, data_loader):
        self.client = client
        self.data_loader = data_loader
        self.intent_classifier = IntentClassifier(client)
        self.embedding_gen = EmbeddingGenerator(client)
        self.rag_retriever = RAGRetriever(data_loader, self.embedding_gen)
        self.response_generator = ResponseGenerator(client, data_loader)
        self.conversation_history = []
    
    def chat(self, user_query: str, verbose: bool = True) -> Dict:
        """Main chat function"""
        
        intent = self.intent_classifier.classify(user_query)
        if verbose:
            print(f" Detected Intent: {intent}")
        
        retrieved_docs = self.rag_retriever.retrieve(user_query, intent, top_k=3)
        if verbose:
            print(f" Retrieved {len(retrieved_docs)} relevant documents")
            for i, doc in enumerate(retrieved_docs, 1):
                print(f"  {i}. {doc['qa'].get('disease', 'N/A')} (score: {doc['score']:.3f})")
        
        response = self.response_generator.generate(user_query, intent, retrieved_docs)
        
        self.conversation_history.append({
            "query": user_query,
            "intent": intent,
            "response": response,
            "retrieved_docs": len(retrieved_docs)
        })
        
        return {
            "query": user_query,
            "intent": intent,
            "response": response,
            "retrieved_docs": retrieved_docs
        }
    
    def reset_conversation(self):
        """Reset conversation history"""
        self.conversation_history = []
        print("✓ Conversation history reset")

In [9]:
import os
import glob

def load_json_files(data_folder: str = "."):
    """
    Load tất cả 6 file JSON từ thư mục chỉ định
    
    Args:
        data_folder: Đường dẫn tới thư mục chứa các file JSON
                    Mặc định là thư mục hiện tại "."
    
    Returns:
        Dictionary chứa content của 6 file dạng JSON string
    """
    
    file_mapping = {
        'disease_names': 'kb_index_disease_name.json',
        'docs': 'kb_index_docs.json',
        'meta': 'kb_index_meta.json',
        'symptom_dict': 'kb_index_symptom_dict.json',
        'qa_flat': 'qa_bank_tieuhoa_flat.json',
        'qa_full': 'qa_bank_tieuhoa.json'
    }
    
    loaded_data = {}
    
    print(f" Loading data from folder: {os.path.abspath(data_folder)}")
    print("="*70)
    
    for key, filename in file_mapping.items():
        filepath = os.path.join(data_folder, filename)
        
        try:
            if not os.path.exists(filepath):
                print(f" File not found: {filename}")
                print(f"   Path: {filepath}")
                continue
            
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
                loaded_data[key] = json.dumps(data, ensure_ascii=False)
            
            data_obj = json.loads(loaded_data[key])
            if isinstance(data_obj, dict):
                print(f"✓ {filename:40s} - {len(data_obj)} entries")
            elif isinstance(data_obj, list):
                print(f"✓ {filename:40s} - {len(data_obj)} items")
            else:
                print(f"✓ {filename:40s} - Loaded successfully")
                
        except json.JSONDecodeError as e:
            print(f" JSON decode error in {filename}: {str(e)}")
        except Exception as e:
            print(f" Error loading {filename}: {str(e)}")
    
    print("="*70)
    
    # Kiểm tra xem đã load đủ 6 file chưa
    missing_files = set(file_mapping.keys()) - set(loaded_data.keys())
    if missing_files:
        print(f"\n  WARNING: Missing files: {missing_files}")
        print("Chatbot sẽ chạy với dữ liệu không đầy đủ!")
    else:
        print(f"\n Successfully loaded all 6 JSON files!")
    
    return loaded_data

In [10]:
try:
    DATA_FOLDER = "." 
   
    
    json_data = load_json_files(DATA_FOLDER)
    
    data_loader.load_from_strings(
        disease_json=json_data.get('disease_names', '{}'),
        docs_json=json_data.get('docs', '[]'),
        meta_json=json_data.get('meta', '{}'),
        symptom_json=json_data.get('symptom_dict', '{}'),
        qa_flat_json=json_data.get('qa_flat', '[]'),
        qa_full_json=json_data.get('qa_full', '[]')
    )
    
    print("\n" + "="*70)
    print(" DATA LOADING SUMMARY")
    print("="*70)
    print(f"Diseases: {len(data_loader.disease_names)}")
    print(f"Documents: {len(data_loader.docs)}")
    print(f"Symptoms: {len(data_loader.symptom_dict)}")
    print(f"QA Flat: {len(data_loader.qa_bank_flat)}")
    print(f"QA Full: {len(data_loader.qa_bank_full)}")
    print("="*70)
    
except Exception as e:
    print(f"\n CRITICAL ERROR loading data: {str(e)}")
    print("\n  Fallback to SAMPLE DATA for testing...")

 Loading data from folder: /Users/thanhhai/Projects1/python/grp/final
✓ kb_index_disease_name.json               - 95 entries
✓ kb_index_docs.json                       - 548 items
✓ kb_index_meta.json                       - 6 entries
✓ kb_index_symptom_dict.json               - 95 entries
✓ qa_bank_tieuhoa_flat.json                - 55 items
✓ qa_bank_tieuhoa.json                     - 11 items

 Successfully loaded all 6 JSON files!
✓ Loaded 95 diseases
✓ Loaded 548 documents
✓ Loaded 55 QA pairs
✓ Loaded 11 disease entries

 DATA LOADING SUMMARY
Diseases: 95
Documents: 548
Symptoms: 95
QA Flat: 55
QA Full: 11


In [11]:
# ===== Initialize Chatbot =====
print("\n Initializing Medical Chatbot...")
chatbot = MedicalChatbot(client, data_loader)
print(" Medical Chatbot initialized successfully!\n")

print(" Quick Statistics:")
print(f"   - Total diseases in index: {len(data_loader.disease_names)}")
print(f"   - Total QA pairs: {len(data_loader.qa_bank_flat)}")
print(f"   - Total disease entries: {len(data_loader.qa_bank_full)}")
print(f"   - Embedding cache size: {len(chatbot.embedding_gen.embedding_cache)}")
print("\n Chatbot is ready to answer questions!")
print("   Run: test_chatbot() or interactive_chat()\n")


 Initializing Medical Chatbot...
Building QA embeddings index...
✓ Built index with 55 QA pairs
 Medical Chatbot initialized successfully!

 Quick Statistics:
   - Total diseases in index: 95
   - Total QA pairs: 55
   - Total disease entries: 11
   - Embedding cache size: 55

 Chatbot is ready to answer questions!
   Run: test_chatbot() or interactive_chat()



In [12]:
def test_chatbot():
    
    test_queries = [
        "dự báo thời tiết hôm nay?",
        "Nghệ vàng có tác dụng gì với viêm dạ dày?",
        "Thuốc Omeprazole dùng như thế nào?",
        "Tôi bị đau bụng, buồn nôn, có thể là bệnh gì?",
        "Ma túy hút có chết k"
    ]
    
    print("=" * 70)
    print("TESTING MEDICAL CHATBOT")
    print("=" * 70)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*70}")
        print(f"TEST {i}: {query}")
        print(f"{'='*70}")
        
        result = chatbot.chat(query, verbose=True)
        
        print(f"\n💬 Response:\n{result['response']}")
        print(f"\n{'-'*70}")

# Run tests
test_chatbot()

TESTING MEDICAL CHATBOT

TEST 1: dự báo thời tiết hôm nay?
 Detected Intent: general
 Retrieved 3 relevant documents
  1. Diarrhea (Tiêu chảy) (score: 0.241)
  2. Diarrhea (Tiêu chảy) (score: 0.233)
  3. Hemorrhoids (Trĩ) (score: 0.211)

💬 Response:
CSDL chưa có thông tin về **dự báo thời tiết** hôm nay. ⚠️ Thông tin mang tính tham khảo, không thay thế chẩn đoán và tư vấn y khoa trực tiếp.

----------------------------------------------------------------------

TEST 2: Nghệ vàng có tác dụng gì với viêm dạ dày?
 Detected Intent: herb
 Retrieved 3 relevant documents
  1. Gastritis (Viêm dạ dày) (score: 0.568)
  2. Gastroesophageal reflux – GERD (Trào ngược dạ dày thực quản) (score: 0.456)
  3. Hepatitis (Viêm gan) (score: 0.392)

💬 Response:
Nghệ vàng có một số tác dụng liên quan đến **viêm dạ dày** (Gastritis) như sau:

- **Thảo dược chính**: Nghệ vàng
- **Cơ chế/đích tác động**:
  - **Curcumin** trong nghệ vàng có tác dụng **kháng viêm**, **chống oxy hóa**, và **bảo vệ niêm mạc** dạ dà

In [13]:
import pickle
import json
from datetime import datetime

def save_chatbot_model(chatbot, data_loader, save_dir="./saved_model"):
    """
    Lưu toàn bộ chatbot model và data để sử dụng sau
    
    Args:
        chatbot: MedicalChatbot instance
        data_loader: DataLoader instance
        save_dir: Thư mục lưu model
    """
    
    os.makedirs(save_dir, exist_ok=True)
    
    print(f"\n{'='*70}")
    print(f" SAVING MEDICAL CHATBOT MODEL")
    print(f"{'='*70}")
    
    print("\n[1/5] Saving Data Loader...")
    data_dict = {
        'disease_names': data_loader.disease_names,
        'docs': data_loader.docs,
        'metadata': data_loader.metadata,
        'symptom_dict': data_loader.symptom_dict,
        'qa_bank_flat': data_loader.qa_bank_flat,
        'qa_bank_full': data_loader.qa_bank_full
    }
    
    with open(os.path.join(save_dir, 'data_loader.json'), 'w', encoding='utf-8') as f:
        json.dump(data_dict, f, ensure_ascii=False, indent=2)
    print("   ✓ Data Loader saved to data_loader.json")
    
    print("\n[2/5] Saving Embedding Cache...")
    with open(os.path.join(save_dir, 'embedding_cache.pkl'), 'wb') as f:
        pickle.dump(chatbot.embedding_gen.embedding_cache, f)
    print(f"   ✓ Cached {len(chatbot.embedding_gen.embedding_cache)} embeddings")
    
    print("\n[3/5] Saving QA Embeddings Index...")
    with open(os.path.join(save_dir, 'qa_embeddings.pkl'), 'wb') as f:
        pickle.dump(chatbot.rag_retriever.qa_embeddings, f)
    print(f"   ✓ Saved {len(chatbot.rag_retriever.qa_embeddings)} QA embeddings")
    
    print("\n[4/5] Saving Conversation History...")
    with open(os.path.join(save_dir, 'conversation_history.json'), 'w', encoding='utf-8') as f:
        json.dump(chatbot.conversation_history, f, ensure_ascii=False, indent=2)
    print(f"   ✓ Saved {len(chatbot.conversation_history)} conversations")
    
    print("\n[5/5] Saving Model Metadata...")
    metadata = {
        'saved_at': datetime.now().isoformat(),
        'total_diseases': len(data_loader.disease_names),
        'total_qa_pairs': len(data_loader.qa_bank_flat),
        'total_embeddings': len(chatbot.embedding_gen.embedding_cache),
        'model_version': '1.0',
        'openai_model': 'gpt-4o-mini',
        'embedding_model': 'text-embedding-3-small'
    }
    
    with open(os.path.join(save_dir, 'model_metadata.json'), 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)
    print("   ✓ Model metadata saved")
    
    print(f"\n{'='*70}")
    print(f" MODEL SAVED SUCCESSFULLY!")
    print(f"{'='*70}")
    print(f" Save location: {os.path.abspath(save_dir)}")
    print(f"\n Summary:")
    print(f"   - Diseases: {metadata['total_diseases']}")
    print(f"   - QA Pairs: {metadata['total_qa_pairs']}")
    print(f"   - Cached Embeddings: {metadata['total_embeddings']}")
    print(f"   - Conversation History: {len(chatbot.conversation_history)}")
    print(f"   - Saved at: {metadata['saved_at']}")
    print(f"{'='*70}\n")
    
    return save_dir


def load_chatbot_model(save_dir="./saved_model"):
    """
    Load chatbot model đã lưu để sử dụng
    
    Args:
        save_dir: Thư mục chứa model đã lưu
        
    Returns:
        chatbot: MedicalChatbot instance đã được load
    """
    
    print(f"\n{'='*70}")
    print(f"📂 LOADING MEDICAL CHATBOT MODEL")
    print(f"{'='*70}")
    
    if not os.path.exists(save_dir):
        raise FileNotFoundError(f"Model directory not found: {save_dir}")
    
    print("\n[1/5] Loading Data Loader...")
    with open(os.path.join(save_dir, 'data_loader.json'), 'r', encoding='utf-8') as f:
        data_dict = json.load(f)
    
    loaded_data_loader = DataLoader()
    loaded_data_loader.disease_names = data_dict['disease_names']
    loaded_data_loader.docs = data_dict['docs']
    loaded_data_loader.metadata = data_dict['metadata']
    loaded_data_loader.symptom_dict = data_dict['symptom_dict']
    loaded_data_loader.qa_bank_flat = data_dict['qa_bank_flat']
    loaded_data_loader.qa_bank_full = data_dict['qa_bank_full']
    print(f"   ✓ Loaded {len(loaded_data_loader.disease_names)} diseases")
    
    print("\n[2/5] Initializing Chatbot...")
    loaded_chatbot = MedicalChatbot(client, loaded_data_loader)
    print("   ✓ Chatbot initialized")
    
    print("\n[3/5] Loading Embedding Cache...")
    with open(os.path.join(save_dir, 'embedding_cache.pkl'), 'rb') as f:
        loaded_chatbot.embedding_gen.embedding_cache = pickle.load(f)
    print(f"   ✓ Loaded {len(loaded_chatbot.embedding_gen.embedding_cache)} cached embeddings")
    
    print("\n[4/5] Loading QA Embeddings Index...")
    with open(os.path.join(save_dir, 'qa_embeddings.pkl'), 'rb') as f:
        loaded_chatbot.rag_retriever.qa_embeddings = pickle.load(f)
    print(f"   ✓ Loaded {len(loaded_chatbot.rag_retriever.qa_embeddings)} QA embeddings")
    
    print("\n[5/5] Loading Conversation History...")
    try:
        with open(os.path.join(save_dir, 'conversation_history.json'), 'r', encoding='utf-8') as f:
            loaded_chatbot.conversation_history = json.load(f)
        print(f"   ✓ Loaded {len(loaded_chatbot.conversation_history)} conversations")
    except:
        print("   ℹ  No conversation history found (starting fresh)")
    
    with open(os.path.join(save_dir, 'model_metadata.json'), 'r', encoding='utf-8') as f:
        metadata = json.load(f)
    
    print(f"\n{'='*70}")
    print(f" MODEL LOADED SUCCESSFULLY!")
    print(f"{'='*70}")
    print(f" Model Info:")
    print(f"   - Version: {metadata['model_version']}")
    print(f"   - Saved at: {metadata['saved_at']}")
    print(f"   - Diseases: {metadata['total_diseases']}")
    print(f"   - QA Pairs: {metadata['total_qa_pairs']}")
    print(f"{'='*70}\n")
    
    return loaded_chatbot



print("\n Saving current chatbot model...")
save_path = save_chatbot_model(chatbot, data_loader, save_dir="./saved_model")

print("\n Testing model loading...")
loaded_chatbot = load_chatbot_model(save_dir="./saved_model")

# Test chatbot đã load
print("\n Testing loaded chatbot with a sample query...")
test_query = "Nghệ vàng có tác dụng gì với viêm dạ dày?"
result = loaded_chatbot.chat(test_query, verbose=False)
print(f"\n Query: {test_query}")
print(f"\n Response:\n{result['response'][:200]}...")

print("\n Model save/load test completed successfully!")
print("\n Usage:")
print("   - To save: save_chatbot_model(chatbot, data_loader)")
print("   - To load: loaded_chatbot = load_chatbot_model()")


 Saving current chatbot model...

 SAVING MEDICAL CHATBOT MODEL

[1/5] Saving Data Loader...
   ✓ Data Loader saved to data_loader.json

[2/5] Saving Embedding Cache...
   ✓ Cached 60 embeddings

[3/5] Saving QA Embeddings Index...
   ✓ Saved 55 QA embeddings

[4/5] Saving Conversation History...
   ✓ Saved 5 conversations

[5/5] Saving Model Metadata...
   ✓ Model metadata saved

 MODEL SAVED SUCCESSFULLY!
 Save location: /Users/thanhhai/Projects1/python/grp/final/saved_model

 Summary:
   - Diseases: 95
   - QA Pairs: 55
   - Cached Embeddings: 60
   - Conversation History: 5
   - Saved at: 2025-12-23T22:30:57.481276


 Testing model loading...

📂 LOADING MEDICAL CHATBOT MODEL

[1/5] Loading Data Loader...
   ✓ Loaded 95 diseases

[2/5] Initializing Chatbot...
Building QA embeddings index...
✓ Built index with 55 QA pairs
   ✓ Chatbot initialized

[3/5] Loading Embedding Cache...
   ✓ Loaded 60 cached embeddings

[4/5] Loading QA Embeddings Index...
   ✓ Loaded 55 QA embeddings

[5/

In [ ]:
import os

save_dir = "./saved_model"
print("Abs path:", os.path.abspath(save_dir))
print("Exists:", os.path.exists(save_dir))

if os.path.exists(save_dir):
    print("Files:")
    for fn in os.listdir(save_dir):
        p = os.path.join(save_dir, fn)
        print("-", fn, "|", os.path.getsize(p), "bytes")
